In [1]:
import tifffile as tiff
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
import os, shutil
import glob
import numpy as np
from pathlib import Path

In [2]:
# activate cellpose-env
# to use GPU, first check on the terminal: 
!nvcc --version 
!nvidia-smi
# then turn use GPU to True

from cellpose import core, utils, io, models, metrics, plot
from cellpose.io import imread
from glob import glob

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:30:10_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0
Fri Jun  6 14:33:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 5000              WDDM  |   00000000:D8:00.0 Off |                  Off |
| 33%

c:\Users\fronte\.conda\envs\cellpose\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>>> GPU activated? YES


membrane and nuclei segmentation

In [ ]:
# CHANGE MODEL_PATH, CHANNELS AND DIAMETER AS REQUIRED
# ADD INPUT FOLDER WHERE CROPPED TRACK IMAGES ARE

# Setup logging for Cellpose
io.logger_setup()

### MODEL
# Custom model path
model_path = "H:/PROJECTS-03/Feyza/segmentation-training/cellpose_nuclei_membrane/training_data_05z/models/CP_20241007_h2bxncad" #membrane (w nucleus)
#model_path = "H:/PROJECTS-03/Agnese/cellpose training/2d/models/nuclei_h2b" #nucleus

model = models.CellposeModel(gpu=True, pretrained_model=model_path)
### PARAMETERS
# Define channels
channels = [[1, 2]]  # cytoplasm: 1, nucleus: 2
#channels = [[3,0]] # only nucleus: if multi-channel image 3=nucleus is the third channel in the image. If there is only the nucleus (gray scale) is [0,0]

# Segmentation parameters
diameter = 25  # in pixels for membrane
#diameter = 17 # in pixels for nucleus
cellprob_threshold = -1

# Input folder containing images
input_folder = Path("E:/Agnese/20250328_gullum_modERK_h2a/20250328_141322_Experiment/quantification/nuclei/")
output_folder = input_folder / "denoised_segmentation_masks_membrane"
output_folder.mkdir(exist_ok=True)  # Create folder if it doesn't exist

# Process all files in the folder
for img_file in input_folder.glob("*.tif"):
    print(f"Processing file: {img_file.name}")

    # Read image
    imgs = imread(img_file)
    print(f'Loaded image {img_file.name} with shape: {imgs.shape} and data type {imgs.dtype}')

    # Generate output file name
    output_file_base = output_folder / img_file.stem  # Use the base name without extension

    # USE cellposeDenoiseModel FOR DENOISING THE IMAGE
    from cellpose import denoise
    dn = denoise.DenoiseModel (model_type='denoise_cyto3', gpu=True, chan2=True)
    imgs = dn.eval(imgs, channels=[1,2], diameter=25.)

    # Perform segmentation
    masks, flows, styles = model.eval(
        imgs,
        diameter=diameter,
        channels=channels,
        cellprob_threshold=cellprob_threshold,
        do_3D=False,
        anisotropy=1.5,
        min_size=-1,
    )

    # Save output to *_seg.npy
    seg_file = str(output_file_base) + "_seg"
    io.masks_flows_to_seg(imgs, masks, flows, seg_file, channels=channels)

    # Save output masks to tiffs/pngs or txt files for ImageJ
    io.save_masks(
        imgs,
        masks,
        flows,
        str(output_file_base),
        channels=channels,
        png=False,  # Save masks as PNGs
        tif=True,  # Save masks as TIFFs
        save_txt=True,  # Save txt outlines for ImageJ
        save_flows=False,  # Save flows as TIFFs
        save_outlines=False,  # Save outlines as TIFFs
        save_mpl=False,  # Make matplotlib fig to view (WARNING: SLOW W/ LARGE IMAGES)
    )

    print(f"Saved results for {img_file.name} to {output_folder}")

print("All files processed.")

2025-03-15 11:42:36,900 [INFO] WRITING LOG OUTPUT TO C:\Users\fronte\.cellpose\run.log
2025-03-15 11:42:36,901 [INFO] 
cellpose version: 	3.0.10 
platform:       	win32 
python version: 	3.9.19 
torch version:  	1.8.2
2025-03-15 11:42:37,003 [INFO] ** TORCH CUDA version installed and working. **
2025-03-15 11:42:37,004 [INFO] >>>> using GPU
2025-03-15 11:42:37,148 [INFO] >>>> loading model H:/PROJECTS-03/Feyza/segmentation-training/cellpose_nuclei_membrane/training_data_05z/models/CP_20241007_h2bxncad
2025-03-15 11:42:37,976 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-03-15 11:42:37,977 [INFO] >>>> model diam_labels =  23.713 (mean diameter of training ROIs)
Processing file: mem000.tif
Loaded image mem000.tif with shape: (2, 636, 1218) and data type uint16
2025-03-15 11:42:38,043 [INFO] >> denoise_cyto3 << model set to be used
2025-03-15 11:42:38,046 [INFO] ** TORCH CUDA version installed and working. **
2025-03-15 11:42:38,047 [INFO] >>>> us

In [3]:
# CHANGE MODEL_PATH, CHANNELS AND DIAMETER AS REQUIRED
# ADD INPUT FOLDER WHERE CROPPED TRACK IMAGES ARE

# Setup logging for Cellpose
io.logger_setup()

### MODEL
# Custom model path
# model_path = "H:/PROJECTS-03/Feyza/segmentation/cellpose_nuclei_membrane/training_data_05z/models/CP_20241007_h2bxncad" #membrane (w nucleus)
model_path = "H:/PROJECTS-03/Agnese/cellpose training/2d/models/nuclei_h2b" #nucleus

model = models.CellposeModel(gpu=True, pretrained_model=model_path)
### PARAMETERS
# Define channels
# channels = [[1, 2]]  # cytoplasm: 1, nucleus: 2
channels = [[0,0]] # only nucleus: if multi-channel image 3=nucleus is the third channel in the image. If there is only the nucleus (gray scale) is [0,0]

# Segmentation parameters
# diameter = 25  # in pixels for membrane
diameter = 17 # in pixels for nucleus
cellprob_threshold = -1

# Input folder containing images
input_folder = Path("E:/Agnese/20250328_gullum_modERK_h2a/20250328_141322_Experiment/quantification/nuclei/")
output_folder = input_folder / "denoised_segmentation_masks_nuclei"
output_folder.mkdir(exist_ok=True)  # Create folder if it doesn't exist

# Process all files in the folder
for img_file in input_folder.glob("*.tif"):
    print(f"Processing file: {img_file.name}")

    # Read image
    imgs = imread(img_file)
    print(f'Loaded image {img_file.name} with shape: {imgs.shape} and data type {imgs.dtype}')

    # Generate output file name
    output_file_base = output_folder / img_file.stem  # Use the base name without extension

    # USE cellposeDenoiseModel FOR DENOISING THE IMAGE
    from cellpose import denoise
    dn = denoise.DenoiseModel (model_type='denoise_cyto3', gpu=True, chan2=True)
    imgs = dn.eval(imgs, channels=[0,0], diameter=17.)

    # Perform segmentation
    masks, flows, styles = model.eval(
        imgs,
        diameter=diameter,
        channels=channels,
        cellprob_threshold=cellprob_threshold,
        do_3D=False,
        anisotropy=1.5,
        min_size=-1,
    )

    # Save output to *_seg.npy
    seg_file = str(output_file_base) + "_seg"
    io.masks_flows_to_seg(imgs, masks, flows, seg_file, channels=channels)

    # Save output masks to tiffs/pngs or txt files for ImageJ
    io.save_masks(
        imgs,
        masks,
        flows,
        str(output_file_base),
        channels=channels,
        png=False,  # Save masks as PNGs
        tif=True,  # Save masks as TIFFs
        save_txt=True,  # Save txt outlines for ImageJ
        save_flows=False,  # Save flows as TIFFs
        save_outlines=False,  # Save outlines as TIFFs
        save_mpl=False,  # Make matplotlib fig to view (WARNING: SLOW W/ LARGE IMAGES)
    )

    print(f"Saved results for {img_file.name} to {output_folder}")

print("All files processed.")

2025-06-06 14:34:53,998 [INFO] WRITING LOG OUTPUT TO C:\Users\fronte\.cellpose\run.log
2025-06-06 14:34:54,000 [INFO] 
cellpose version: 	3.0.10 
platform:       	win32 
python version: 	3.9.19 
torch version:  	1.8.2
2025-06-06 14:34:54,160 [INFO] ** TORCH CUDA version installed and working. **
2025-06-06 14:34:54,162 [INFO] >>>> using GPU
2025-06-06 14:34:54,324 [INFO] >>>> loading model H:/PROJECTS-03/Agnese/cellpose training/2d/models/nuclei_h2b
2025-06-06 14:34:55,457 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-06-06 14:34:55,459 [INFO] >>>> model diam_labels =  14.500 (mean diameter of training ROIs)
Processing file: nuclei0000.tif
Loaded image nuclei0000.tif with shape: (972, 1512) and data type uint16
2025-06-06 14:34:55,507 [INFO] >> denoise_cyto3 << model set to be used
2025-06-06 14:34:55,509 [INFO] ** TORCH CUDA version installed and working. **
2025-06-06 14:34:55,511 [INFO] >>>> using GPU
2025-06-06 14:34:55,890 [INFO] >>>> mode